# BTC Solar Solver


In [1]:
import BTCminer

# PV System Info
pv_system = {'location':{
		'lat':38.897957,
		'lon':-77.036560},
	'specs':{
		'array_type':1, # Fixed axis
		'azimuth':180,
		'tilt':28.7,
		'losses':1}}

# NREL API
nrel_api_key = 'API'
nrel_api_email= 'name@email.com'

# Initialize Miner
antminer_s9 = BTCminer.BTCminer(13.5, 1323)
antminer_s9.fit_solar(pv_system, nrel_api_key, nrel_api_email, 100)

OSolving for Optimal Solar System
Target kWh: 11589.48

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: /home/carlos/Documents/GitHub/BTCsolar/data/PySAM Downloaded Weather Files/nsrdb_38.897957_-77.03656_psm3-tmy_60_tmy.csv

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: /home/carlos/Documents/GitHub/BTCsolar/data/PySAM Downloaded Weather Files/nsrdb_38.897957_-77.03656_psm3-tmy_60_tmy.csv
Optimal Solar Capacity (kW): ac=4:dc=11
Output with best parameters: 11732.14413535751


(4, 11)